# Install necessary packages

In [ ]:
%pip install transformers accelerate

In [ ]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Load model

In [ ]:
# load model from huggingface
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, StoppingCriteria

# using the second GPU only
tokenizer = AutoTokenizer.from_pretrained("felipeoes/cocoruta-7b", device_map="cuda:1")
model = AutoModelForCausalLM.from_pretrained("felipeoes/cocoruta-7b", device_map="cuda:1", torch_dtype=torch.float16)

model

In [9]:
# Define an early stopping ccriteria
class StopOnString(StoppingCriteria):
    def __init__(self, target_sequence, prompt):
        self.target_sequence = target_sequence
        self.prompt=prompt

    def __call__(self, input_ids, scores, **kwargs):
        # Get the generated text as a string
        generated_text = tokenizer.decode(input_ids[0])
        generated_text = generated_text.replace(self.prompt,'')
        # Check if the target sequence appears in the generated text
        if self.target_sequence in generated_text:
            return True  # Stop generation

        return False  # Continue generation

# INFERENCE
stop_string = "###"
streamer = TextStreamer(tokenizer, skip_prompt=True)

input_text = "### Pergunta: O que é a Amazônia Azul?\n### Resposta:"
input_ids = tokenizer(input_text, return_tensors='pt').input_ids.to(model.device)
_ = model.generate(input_ids,
                            streamer=streamer,
                            pad_token_id=tokenizer.eos_token_id, 
                            max_length=256, 
                            temperature=0.8,
                            top_p=0.9,
                            top_k=30,
                            stopping_criteria=[StopOnString(stop_string, input_text)],
                            do_sample=True,
                            num_return_sequences=1,
)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



A Amazônia Azul é a região marítima brasileira que compreende a Zona Econômica Exclusiva (ZEE) e a Plataforma Continental. A ZEE é a área do mar adjacente ao território do Brasil, com largura de até 200 milhas marítimas, e a Plataforma Continental é a extensão do território brasileiro sob o mar, até o limite de 200 metros de profundidade ou até o limite de 350 milhas marítimas, se a profundidade for menor que 200 metros.

###
